# Meetup #13 – Jeudi 28 novembre 2019

## Agenda

- tour de table
- présentations courtes
- activités

## Présentations

- Christophe : cartographie avec Python et autres
    - [QGIS](https://qgis.org/)
    - https://towardsdatascience.com/mapping-geograph-data-in-python-610a963d2d7f
    - https://chrishavlin.com/2016/11/16/shapefiles-tutorial/
    - [Seaborn](http://seaborn.pydata.org/) : surcouche de haut niveau à `matplotlib`
- Christophe : comment je fais pour distribuer des applications
   - création d'applications Windows (.exe) avec [pyinstaller](http://www.pyinstaller.org/)
   - autres outils du même genre : [cx_Freeze](https://cx-freeze.readthedocs.io/en/latest/), [py2exe](http://www.py2exe.org/)...
- Bruno : retour d'expérience sur [poetry](https://poetry.eustace.io/)

## Activités

- par où commencer ?
- les exceptions
- ~~les imports~~


### Par où commencer ?

Il y a beaucoup de ressources, y compris gratuites, sur le web. C'est un sujet que l'on avait déja abordé dans des précédents meetups : cf. notes du [meetup #6](https://github.com/LaTechAmienoise/Meetup-Python/blob/master/20181115/Meetup.ipynb) ou du [meetup #7](https://github.com/LaTechAmienoise/Meetup-Python/blob/master/20181211/Meetup%207.ipynb). 

### Les exceptions

Les exceptions sont un mécanisme de gestion d'erreurs qui permet de séparer l'endroit où le problème est détecté (là où on lance l'exception) et l'endroit où on est en mesure de le traiter ou d'y réagir (là où on attrape l'exception). 

In [1]:
def calcule_un_truc(age):
    if age < 0:
        raise ValueError("l'âge ne doit pas être négatif")
    return age * 42

Un bloc `try` avec une clause `except` permet d'intercepter (toutes) les erreurs.

In [2]:
try:
    calcule_un_truc(-1)
except:
    print("erreur")

erreur


Lorsque l'on s'intéresse à un type d'erreur particulier (ici `ValueError`), c'est mieux d'être précis. On peut mettre plusieurs clauses `except`, et terminer par une clause « attrape tout » :

In [3]:
try:
    calcule_un_truc(-1)
except ValueError:
    print("oups, mauvaise valeur")
except:  # toutes les autres erreurs
    print("erreur inattendue")

oups, mauvaise valeur


Si on veut examiner l'objet exception (qui peut porter des informations utiles), on peut utiliser le mot-clé `as` pour lui associer un nom :

In [4]:
try:
    calcule_un_truc(-1)
except ValueError as exc:
    print(f"erreur : {exc}")

erreur : l'âge ne doit pas être négatif


On peut définir ses propres classes d'exceptions :

In [5]:
class AgeInvalide(ValueError):
    pass

class AnneeInvalide(ValueError):
    pass

def calcule_un_truc(age, annee):
    if age < 0:
        raise AgeInvalide
    if annee > 2019:
        raise AnneeInvalide
    return age * 42

In [6]:
try:
    calcule_un_truc(-3, 2019)
except AgeInvalide:
    print("Âge invalide")
except AnneeInvalide:
    print("Année invalide")

Âge invalide


In [7]:
try:
    calcule_un_truc(-3, 2019)
except ValueError:  # attrape aussi les classes dérivées
    print("Âge ou année invalide")

Âge ou année invalide


La clause `finally` permet d'exécuter du code dans tous les cas, aussi bien quand le bloc `try` s'est exécuté avec succès que quand une exception a eu lieu.

In [8]:
try:
    print("ok")
finally:
    print("pour terminer")

ok
pour terminer


In [9]:
try:
    1 / 0
    print("ce code n'est jamais exécuté")
except ZeroDivisionError:
    print("erreur")
finally:
    print("pour terminer")

erreur
pour terminer


C'est utile pour mettre du code de « nettoyage » : fermer un fichier, libérer un verrou, etc.

In [10]:
f = open("fichier.txt")
try:
    f.read()
except:
    f.close()

Dans beaucoup de cas on gagnera cependant à utiliser un *context manager*, qui s'occupera de ça pour nous automatiquement :

In [11]:
with open("fichier.txt") as f:
    f.read()

Pour créer son propre *context manager*, on définit une classe qui implémente le protocole suivant :

In [12]:
class MonTruc:
    def __enter__(self):
        print("avant")
    def __exit__(self, *exc):
        print("après")

In [13]:
with MonTruc():
    print("OK")

avant
OK
après


In [14]:
try:
    with MonTruc():
        1/0
except:
    pass

avant
après


On peut aussi l'écrire sous forme d'un générateur (une fonction avec un `yield`) grâce à un décorateur spécial fourni dans la bibliothèque standard :

In [15]:
from contextlib import contextmanager

@contextmanager
def montruc():
    print("avant")
    try:
        yield
    finally:
        print("après")

In [16]:
with montruc():
    print("OK")

avant
OK
après


In [17]:
try:
    with montruc():
        1/0
except:
    pass

avant
après
